In [2]:
import torch,shutil, random, os
import os.path as osp
import numpy as np
import pandas as pd
from plot_helper import find_files,create_new_dir
random.seed(0)

In [3]:
'''
给每个bag设立一个单独的文件夹,方便后面测试使用
'''
bag_dirpath = '/home/jiang/trajectory_pred/ld_dataset/Dataset_for_Master_Thesis'

def move_bag_to_dir(bag_dirpath):
    bags = find_files(bag_dirpath)
    for b in bags:
        bag_name = b.split('/')[-1][:-4]
        dst_dir = create_new_dir(bag_dirpath,bag_name)
        dst_path = osp.join(dst_dir,b.split('/')[-1])
        shutil.move(b, dst_path)
        
# move_bag_to_dir(bag_dirpath)

In [4]:
'''
给每个bag都生成对应的obj以及ego.csv
'''
import subprocess
from tqdm import tqdm
bag2csv_path = "/home/jiang/VW/toolchain/compass_script/tools/bag2csv"
def run_bag2csv(_cwd, bag_path, output_dir):
    subprocess.check_output(['./bag2csv', bag_path, output_dir], cwd=_cwd, stderr=subprocess.STDOUT)


def run_bag2csv_all(_cwd, bag_dir):
    bags = find_files(bag_dir,recursive=True)
    for b in tqdm(bags):
        out_path = osp.dirname(b)
        run_bag2csv(_cwd, b, out_path)

# run_bag2csv_all(bag2csv_path,bag_dirpath)

In [83]:
'''
1. 删除没有在0-100m出现过的obj
'''
obj_csv = '/home/jiang/trajectory_pred/ld_dataset/Dataset_for_Master_Thesis/LIDAR_LJ02766_20210916_054052_G260-PDX-006-001-052_000000-000012_LD_final__OD_MERGE_OPP/LIDAR_LJ02766_20210916_054052_G260-PDX-006-001-052_000000-000012_LD_final__OD_MERGE_OPP_obj.csv'
ego_csv = '/home/jiang/trajectory_pred/ld_dataset/Dataset_for_Master_Thesis/LIDAR_LJ02766_20210916_054052_G260-PDX-006-001-052_000000-000012_LD_final__OD_MERGE_OPP/LIDAR_LJ02766_20210916_054052_G260-PDX-006-001-052_000000-000012_LD_final__OD_MERGE_OPP_ego.csv'

obj_df = pd.read_csv(obj_csv)
ego_df = pd.read_csv(ego_csv)

def del_obj_out_range(df,min_dist=0,max_dist=90):
    '''
    删除最远距离小于min_dist或者最小距离大于max_dist的obj
    '''
    # src_obj_id = df['obj_id'].unique()
    src_obj_id = df['object.id'].unique()
    ret_df = pd.DataFrame()
    
    for obj_id in src_obj_id:
        # obj_id_df = df[df.obj_id == obj_id]
        # _min_dist = min(obj_id_df['LocalX'])
        # _max_dist = max(obj_id_df['LocalX'])
        
        obj_id_df = df[df['object.id'] == obj_id]
        _min_dist = min(obj_id_df['pose.position.x'])
        _max_dist = max(obj_id_df['pose.position.x'])
        
        if _min_dist > max_dist or _max_dist < min_dist:
            print(f"delete: The obj id: {obj_id:3} has min_dist {_min_dist:6,.2f} and max_dist {_max_dist:6,.2f}")
        else:
            ret_df = ret_df.append(obj_id_df)
    ret_df.sort_values(by=['object list No.','object.id'],inplace=True)
    return ret_df

del1_obj_df = del_obj_out_range(obj_df)


delete: The obj id:   2 has min_dist 147.22 and max_dist 149.54
delete: The obj id:   4 has min_dist 117.66 and max_dist 154.28
delete: The obj id:   6 has min_dist 111.45 and max_dist 134.72
delete: The obj id:  14 has min_dist  -9.58 and max_dist  -0.83
delete: The obj id:  20 has min_dist 129.00 and max_dist 133.49
delete: The obj id:  22 has min_dist 145.02 and max_dist 146.17
delete: The obj id:  24 has min_dist 126.35 and max_dist 126.35
delete: The obj id:  26 has min_dist 121.01 and max_dist 128.92
delete: The obj id:  28 has min_dist 142.20 and max_dist 144.67
delete: The obj id:  32 has min_dist 110.21 and max_dist 114.33
delete: The obj id:  36 has min_dist 161.59 and max_dist 184.04
delete: The obj id:  38 has min_dist 143.79 and max_dist 144.32
delete: The obj id:  40 has min_dist 138.76 and max_dist 142.27
delete: The obj id:  84 has min_dist 149.13 and max_dist 203.62
delete: The obj id:  94 has min_dist 157.11 and max_dist 203.51
delete: The obj id:  96 has min_dist 181

In [84]:
'''
1. 删除生命周期小于3s的obj-->
'''

def del_obj_life_less(df,duration=3):
    '''
    删除生命周期短于duration的obj
    '''
    # src_obj_id = df['obj_id'].unique()
    src_obj_id = df['object.id'].unique()
    ret_df = pd.DataFrame()
    
    for obj_id in src_obj_id:
        # obj_id_df = df[df.obj_id == obj_id]
        obj_id_df = df[df['object.id'] == obj_id]
        # _duration = max(obj_id_df['timestamp']) - min(obj_id_df['timestamp'])
        _duration = max(obj_id_df['bag timestamp']) - min(obj_id_df['bag timestamp'])
        if _duration < duration:
            print(f"delete: The obj id: {obj_id:3} has duration {_duration:6,.2f}s.")
        else:
            ret_df = ret_df.append(obj_id_df)
    # ret_df.sort_values(by=['frame_id','obj_id'],inplace=True)
    ret_df.sort_values(by=['object list No.','object.id'],inplace=True)
    return ret_df

del2_obj_df = del_obj_life_less(del1_obj_df)

delete: The obj id:   9 has duration   0.50s.
delete: The obj id:  10 has duration   1.30s.
delete: The obj id:  16 has duration   0.35s.
delete: The obj id:  30 has duration   0.25s.
delete: The obj id:  48 has duration   0.65s.
delete: The obj id:  50 has duration   0.55s.
delete: The obj id:  56 has duration   2.20s.
delete: The obj id:  57 has duration   0.25s.
delete: The obj id: 112 has duration   2.40s.
delete: The obj id: 206 has duration   1.90s.


In [89]:
'''
检查是否有重复id
'''
from datetime import datetime
from collections import defaultdict
def dup_id_check(df,gap=0.3):
    '''
    检查是否存在id的间隔大于gap的值
    '''
    # src_obj_id = df['obj_id'].unique()
    src_obj_id = df['object.id'].unique()
    ret_df = pd.DataFrame()
    large_gap = defaultdict(list)
    for obj_id in src_obj_id:
        # obj_id_df = df[df.obj_id == obj_id]
        # ts = obj_id_df['timestamp'].to_numpy()
        # _date = obj_id_df['date'].to_numpy()
        obj_id_df = df[df['object.id'] == obj_id]
        ts = obj_id_df['bag timestamp'].to_numpy()
        f = obj_id_df['object list No.'].to_numpy()
        # _date = obj_id_df['date'].to_numpy()
        _gap = ts[1:] - ts[:-1]
        idxs = np.argwhere(_gap>gap)
        if len(idxs)>0:
            for i in idxs:
                t1 = ts[i[0]]
                t2 = ts[i[0]+1]
                f1,f2 = f[i[0]],f[i[0]+1]
                large_gap[obj_id].append([f1,f2])
                print(f"The obj id: {obj_id:3} has gap of {t2-t1:4,.2f}s ({f2-f1:2} frame) at  {datetime.utcfromtimestamp(t1)} ({t1:.3f}).")
    return large_gap
large_gap = dup_id_check(del2_obj_df)
print(large_gap)

The obj id:  11 has gap of 0.35s ( 7 frame) at  2021-09-16 05:42:12.077146 (1631770932.077).
The obj id:  54 has gap of 0.35s ( 7 frame) at  2021-09-16 05:41:45.877400 (1631770905.877).
defaultdict(<class 'list'>, {11: [[1582, 1589]], 54: [[1058, 1065]]})


In [82]:
'''
检查出现重复id的情况
通过用ld_editor查看：
11以及54-->漏标了6帧
'''
pass

In [93]:
'''
把漏标的给插值上去，只用插值：'pose.position.x','pose.position.y'
直接继承：'object.id'，'class_label_pred'，'dimension_x', 'dimension_y','yaw'
'''

def linear_interpol(df,gap_dict):
    '''
    把gap值较大的id的obj进行线性插值
    '''
    for obj_id,gap_list in gap_dict.items():
        obj_id_df = df[df['object.id'] == obj_id]
        for gap in gap_list:
            start_frame_id,end_frame_id = gap[0],gap[1]
            gap_frames = end_frame_id - start_frame_id -1 
            
            start_frame,end_frame = obj_id_df[obj_id_df['object list No.']==start_frame_id],obj_id_df[obj_id_df['object list No.']==end_frame_id]
            start_x,end_x = start_frame['pose.position.x'],end_frame['pose.position.x']
            start_y,end_y = start_frame['pose.position.y'],end_frame['pose.position.y']
            start_t,end_t = start_frame['bag timestamp'],end_frame['bag timestamp']
            
            x  = np.linspace(start_x,end_x,num=gap_frames+2)[1:-1]
            y = np.linspace(start_y,end_y,num=gap_frames+2)[1:-1]
            t = np.linspace(start_t,end_t,num=gap_frames+2)[1:-1]
            gap_frame_ids = [i for i in range(start_frame_id+1,end_frame_id)]
            for _x,_y,_t,frame_id in zip(x,y,t,gap_frame_ids):
                new_frame = start_frame.copy()
                new_frame['pose.position.x'] = _x
                new_frame['pose.position.y'] = _y
                new_frame['object list No.'] = frame_id
                new_frame['bag timestamp'] = _t
                new_frame['header timestamp'] = _t
                df = df.append(new_frame)
    df.sort_values(by=['object list No.','object.id'],inplace=True)
    return df

interp_df = linear_interpol(del2_obj_df,large_gap)
interp_df.to_csv('/home/jiang/trajectory_pred/ld_dataset/Dataset_for_Master_Thesis/LIDAR_LJ02766_20210916_054052_G260-PDX-006-001-052_000000-000012_LD_final__OD_MERGE_OPP/obj_filter_interp.csv',index = False)
            

In [21]:
'''
把obj_df以及ego_df合并起来，并且算出GlobalX，以及GlobalY
选择其中一个bag作为测试
'''
from datetime import datetime



def cal_globalXY(merge_df):
    '''
    Calculate global X and Y
    '''
    frame_id_set = merge_df['frame_id'].unique()
    for frame_id in frame_id_set:
        frame_id_mask = (merge_df['frame_id']==frame_id)
        obj_id_mask = merge_df['obj_id'] != 0
        
        ego_global_x,ego_global_y = merge_df[frame_id_mask].iloc[0]['GlobalX'],merge_df[frame_id_mask].iloc[0]['GlobalY']
        merge_df.loc[frame_id_mask & obj_id_mask,['GlobalX']] = ego_global_x
        merge_df.loc[frame_id_mask & obj_id_mask,['GlobalY']] = ego_global_y
        
    merge_df.loc[:,'GlobalX'] += merge_df['LocalX']
    merge_df.loc[:,'GlobalY'] += merge_df['LocalY']
    return merge_df
    

def merge_obj_ego(obj_df,ego_df):
    '''
    把obj_df以及ego_df合并起来，并且算出GlobalX，以及GlobalY
    '''
    
    # 取出需要的列
    obj_df = obj_df[['object list No.','bag timestamp','object.id','class_label_pred','pose.position.x','pose.position.y']].copy()
    ego_df = ego_df[['frame seq','bag timestamp','position x','position y']].copy()

    # 给ego_df的列重命名以及赋初值
    ego_df.rename(columns={'frame seq':'frame_id','bag timestamp':'timestamp','position x':'GlobalX','position y':'GlobalY'},inplace=True)
    ego_df['class'] = 'Car'
    ego_df.loc[:,['LocalX','LocalY','obj_id']] = 0

    # 给obj_df的列重命名以及赋初值
    obj_df.rename(columns={'object list No.':'frame_id','bag timestamp':'timestamp','object.id':'obj_id','class_label_pred':'class','pose.position.x':'LocalX','pose.position.y':'LocalY'},inplace=True)
    obj_df.loc[:,['GlobalX','GlobalY']] = 0

    # 把两个df concate起来，并且按照frame_id和obj_id排序，列重新排列,增加一个可读时间
    merge_df = pd.concat([ego_df,obj_df])
    merge_df.sort_values(by=['frame_id','obj_id'],inplace=True)
    merge_df = merge_df[['frame_id','timestamp','obj_id','class','LocalX','LocalY','GlobalX','GlobalY']]
    # merge_df['date'] = pd.to_datetime(merge_df['timestamp'],unit='s').dt.tz_localize('Europe/Berlin')
    merge_df['date'] = pd.to_datetime(merge_df['timestamp'],unit='s')
    merge_df = merge_df[['frame_id','date','timestamp','obj_id','class','LocalX','LocalY','GlobalX','GlobalY']]
    # 计算golbal x y
    merge_df = cal_globalXY(merge_df)
    return merge_df
    
merge_df = merge_obj_ego(obj_df,ego_df)

merge_df.to_csv('/home/jiang/trajectory_pred/ld_dataset/Dataset_for_Master_Thesis/LIDAR_LJ02766_20210916_054052_G260-PDX-006-001-052_000000-000012_LD_final__OD_MERGE_OPP/merge.csv',index = False)

In [24]:
'''
现在的数据是20Hz的，降采样为10Hz
'''
def down_sample(df,r=2):
    src_frame_id = df['frame_id'].unique()
    dst_frame_id = np.arange(start=0,stop=len(src_frame_id),step=r)
    
    ret_df = pd.DataFrame()
    for idx,frame_id in enumerate(dst_frame_id):
        frame_id_df = df[df['frame_id']==frame_id].copy()
        frame_id_df.loc[:,'frame_id'] = idx
        ret_df = ret_df.append(frame_id_df)
    return ret_df

merge_ds_df = down_sample(merge_df)
merge_ds_df.to_csv('/home/jiang/trajectory_pred/ld_dataset/Dataset_for_Master_Thesis/LIDAR_LJ02766_20210916_054052_G260-PDX-006-001-052_000000-000012_LD_final__OD_MERGE_OPP/merge_10hz.csv',index = False)